In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dados = pd.read_csv("../input/video-game-sales-and-ratings/Video_Game_Sales_as_of_Jan_2017.csv")
dados.head()

In [ ]:
dados.isnull().sum()

In [ ]:
dados.drop(['Global_Sales', 'Other_Sales'], axis=1, inplace=True)

In [ ]:
dados.dropna(axis=0, inplace=True)

In [ ]:
dados = dados.loc[dados['NA_Sales'] > 1]
dados = dados.loc[dados['EU_Sales'] > 1]

In [ ]:
nome = dados.Name
dados.drop('Name', axis=1, inplace=True)

In [ ]:
dados.head()

In [ ]:
dados.shape

In [ ]:
venda_na = dados.iloc[:,4].values
venda_jp = dados.iloc[:,6].values
venda_eu = dados.iloc[:,5].values

In [ ]:
previsores = dados.iloc[:,[0,1,2,3,7,8,9,10,11]].values

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

ct = ColumnTransformer([('columns', OneHotEncoder(), [0,2,3,8])], remainder = 'passthrough')
previsores = ct.fit_transform(previsores).toarray()

In [ ]:
previsores.shape

In [ ]:
from keras.layers import Dense, Input, Activation, Dropout
from keras.models import Model

In [ ]:
print((previsores.shape[1] + 3)/2)

In [ ]:
camada_entrada = Input(shape=(61,))
camada_oculta1 = Dense(units=32, activation='sigmoid')(camada_entrada)
camada_oculta2 = Dense(units=32, activation='sigmoid')(camada_oculta1)
camada_saida1 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida2 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units=1, activation='linear')(camada_oculta2)

regressor = Model(inputs=camada_entrada,
                  outputs=[camada_saida1, camada_saida2, camada_saida3])
regressor.compile(optimizer='adam', loss='mse')
regressor.fit(previsores, [venda_eu, venda_jp, venda_na], batch_size=100, epochs=5000)

In [ ]:
previsao_eu, previsao_jp, previsao_na = regressor.predict(previsores)

In [ ]:
print("Média previsão EU:", previsao_eu.mean(), "Média valores reais EU:", venda_eu.mean())

In [ ]:
print("Média previsão JP:", previsao_jp.mean(), "Média valores reais JP:", venda_jp.mean())

In [ ]:
print("Média previsão NA:", previsao_na.mean(), "Média valores reais NA:", venda_na.mean())